In [2]:
from google.cloud import bigquery
client = bigquery.Client()
dataset_ref = client.dataset("world_bank_intl_education", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)
table_ref = dataset_ref.table("international_education")
table = client.get_table(table_ref)
client.list_rows(table, max_results=5).to_dataframe()

C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,country_name,country_code,indicator_name,indicator_code,value,year
0,Chad,TCD,"Enrolment in lower secondary education, both s...",UIS.E.2,321921.0,2012
1,Chad,TCD,"Enrolment in upper secondary education, both s...",UIS.E.3,68809.0,2006
2,Chad,TCD,"Enrolment in upper secondary education, both s...",UIS.E.3,30551.0,1999
3,Chad,TCD,"Enrolment in upper secondary education, both s...",UIS.E.3,79784.0,2007
4,Chad,TCD,"Repeaters in primary education, all grades, bo...",UIS.R.1,282699.0,2006


In [3]:
query = """
    SELECT country_name, AVG(value) as avg_ed_spending_pct
    FROM `bigquery-public-data.world_bank_intl_education.international_education`
    WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' AND year >= 2010 AND year <= 2017
    GROUP BY country_name
    ORDER BY avg_ed_spending_pct DESC
"""
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)
international_education = query_job.to_dataframe()
international_education.head()

,country_name,avg_ed_spending_pct
0,Cuba,12.837270
1,"Micronesia, Fed. Sts.",12.467750
2,Solomon Islands,10.001080
3,Moldova,8.372153
4,Namibia,8.349610


In [5]:
query = """
    SELECT indicator_code, indicator_name, COUNT(1) as num_rows
    FROM `bigquery-public-data.world_bank_intl_education.international_education`
    GROUP BY indicator_code, indicator_name
    HAVING COUNT(1) >= 175
    ORDER BY COUNT(1) DESC
"""
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)
indicator_counts = query_job.to_dataframe()
indicator_counts.head()

,indicator_code,indicator_name,num_rows
0,SP.POP.TOTL,"Population, total",11155
1,SP.POP.GROW,Population growth (annual %),11149
2,SP.POP.1564.TO.ZS,"Population, ages 15-64 (% of total)",10243
3,SP.POP.0014.TO.ZS,"Population, ages 0-14 (% of total)",10233
4,SP.POP.TOTL.FE.ZS,"Population, female (% of total)",10233
